In [70]:
# paras
dataset = 'sports'
dataset_text = 'Sports'
# history = 10

In [71]:
import random
import json
import numpy as np

input data
--------------

In [72]:
f = open(f'../../llm_results/{dataset}/datamaps.json', 'r')
content = f.read()
data_maps = json.loads(content)
print(data_maps.keys())

dict_keys(['user2id', 'item2id', 'id2user', 'id2item', 'attribute2id', 'id2attribute', 'attributeid2num'])


In [73]:
file = open(f"../../llm_results/{dataset}/sequential_data.txt", 'r')
test_samples = []
for line in file.readlines():
    test_samples.append(line[:-1])
file.close()
print(len(test_samples))
test_samples = [x.split(' ') for x in test_samples]
# 1: user_id; 2-n: user_history item_id

35598


In [74]:
file = open(f"../../llm_results/{dataset}/negative_samples.txt", 'r')
negative_samples = []
for line in file.readlines():
    negative_samples.append(line[:-1])
file.close()
print(len(negative_samples))
negative_samples = [x.split(' ') for x in negative_samples]
# 1: user_id; 2-n: negative sample item_id

35598


In [75]:
file = open(f"../../llm_results/{dataset}/meta.json", 'r')
metas = json.load(file)
metas = metas.split('\n')[:-1]
metas = [eval(x) for x in metas]
for item in metas:
    try:
        _ = (item['asin'], item['title'])
    except:
        item['title'] = 'N\A'
        
item_asin2name = [(x['asin'], x['title']) for x in metas]
item_asin2name = {key: value for key, value in item_asin2name}
# item_title2categories = [(x['title'], x['categories'][0][1:]) for x in metas]
# item_title2categories = {key: value for key, value in item_title2categories}

def item_id2title(x):
    asin = data_maps['id2item'][x]
    return item_asin2name[asin]
item_id2name = [(x, item_id2title(x)) for x in data_maps['id2item'].keys()]
item_id2name = {key: value for key, value in item_id2name}

In [76]:
name2item_id = [(item_asin2name[data_maps['id2item'][x]], x) for x in data_maps['id2item'].keys()]
name2item_id = {key: value for key, value in name2item_id}

In [77]:
username_path = f'../../llm_results/{dataset}/user_id2name.pkl'
username = np.load(username_path, allow_pickle=True)

generate prompt from templates
-----------

In [53]:
# template.txt / template_wohistory.txt
# zero-shot instruction construction (history/candidate shuffled)
# template_user_profile.txt / template_profile_history.txt
# zero-shot instruction construction with user profile (without user history)
def generate_zero_shot_prompt(user_id, prompt, dataset_name, his_len, cand_len, his_shuffle=False, cand_shuffle=True, cand_list_id=[]):
    a_user_history = test_samples[int(user_id)-1][1:-1]
    a_user_history = [item_id2name[x] for x in a_user_history]
    if len(a_user_history) > his_len:
        a_user_history = a_user_history[-his_len:]
    # whether to random shuffle user history
    if his_shuffle:
        random.shuffle(a_user_history)
    if len(cand_list_id) == 0:
        cand_list_id = [test_samples[int(user_id)-1][-1]] + negative_samples[int(user_id)-1][1:cand_len]
        # whether to random shuffle candidate list
        if cand_shuffle:
            random.shuffle(cand_list_id)
    cand_list = [item_id2name[x] for x in cand_list_id]
    target = item_id2name[test_samples[int(user_id)-1][-1]]
    rtn_prompt = prompt.replace('{user_name}', username[str(user_id)])
    rtn_prompt = rtn_prompt.replace('{user_history}', str(a_user_history))
    rtn_prompt = rtn_prompt.replace('{candidate_list}', str(cand_list))
    rtn_prompt = rtn_prompt.replace('{dataset}', str(dataset_name))
    rtn_prompt = rtn_prompt.replace('\"', '\'')
    return rtn_prompt, target, cand_list_id

In [54]:
# template_user_profile_generate.txt
# user profile generation
def generate_user_profile_prompt(user_id, prompt, dataset_name, his_len):
    a_user_history = test_samples[int(user_id)-1][1:-1]
    a_user_history = [item_id2name[x] for x in a_user_history]
    if len(a_user_history) > his_len:
        a_user_history = a_user_history[-his_len:]
    target = item_id2name[test_samples[int(user_id)-1][-1]]
    rtn_prompt = prompt.replace('{user_name}', username[str(user_id)])
    rtn_prompt = rtn_prompt.replace('{user_history}', str(a_user_history))
    rtn_prompt = rtn_prompt.replace('{dataset}', str(dataset_name))
    rtn_prompt = rtn_prompt.replace('\"', '\'')
    return rtn_prompt, target

In [55]:
# template_in_context.txt
# in-context learning instructions (last item reinforced)
def generate_in_context_prompt(user_id, prompt, dataset_name, his_len, cand_len, his_shuffle=False, cand_shuffle=True, cand_list_id=None):
    a_user_history = test_samples[int(user_id)-1][1:-1]
    a_user_history = [item_id2name[x] for x in a_user_history]
    if len(a_user_history) > his_len:
        a_user_history = a_user_history[-his_len:]
    # whether to random shuffle user history
    if his_shuffle:
        random.shuffle(a_user_history)
    if cand_list_id == None:
        cand_list_id = [test_samples[int(user_id)-1][-1]] + negative_samples[int(user_id)-1][1:cand_len]
        # whether to random shuffle candidate list
        if cand_shuffle:
            random.shuffle(cand_list_id)
    cand_list = [item_id2name[str(x)] for x in cand_list_id]
    target = item_id2name[test_samples[int(user_id)-1][-1]]
    rtn_prompt = prompt.replace('{user_name}', username[str(user_id)])
    rtn_prompt = rtn_prompt.replace('{user_history}', str(a_user_history))
    rtn_prompt = rtn_prompt.replace('{last_history}', str(a_user_history[:-1]))
    rtn_prompt = rtn_prompt.replace('{last_item}', "[\'"+str(a_user_history[-1])+"\']")
    rtn_prompt = rtn_prompt.replace('{candidate_list}', str(cand_list))
    rtn_prompt = rtn_prompt.replace('{dataset}', str(dataset_name))
    rtn_prompt = rtn_prompt.replace('\"', '\'')
    return rtn_prompt, target, cand_list_id

In [56]:
def read_template(template_file):
    file = open(template_file, 'r')                                      
    templates = []
    for line in file.readlines():
        templates.append(line[:-1])
    file.close()
    return templates

In [57]:
def generate_demonstration_prompts(user_ids, target_user, template_num, dataset_name, history):
    demo_templates = read_template('./templates/template_demonstration.txt')
    example_prompts = []
    for user_id in user_ids:
        prompt_str = "Here\'s a demonstration example:\n"
        demo, target, _ = generate_zero_shot_prompt(str(user_id), demo_templates[template_num], dataset_name, history, 20)
        prompt_str += demo + "\n"
        prompt_str = prompt_str.replace('{target}', "[\'"+target+"\']")
        # prompt_str += "Do you think the pattern of the demonstration example is helpful for you to generate more accurate recommendation for {user}? If it is helpful, make some changes to improve your top-10 recommendation list to user {user}. If you don't think the example is helpful, then just keep the original answer. Still remember that the output format should be a python list of items' titles."
        prompt_str += "Please try to make some changes to improve the recommendation to {user}. Still remember that the output format should be a python list of items' titles."
        prompt_str = prompt_str.replace('{user}', username[target_user])
        example_prompts.append(prompt_str)
    return example_prompts

In [58]:
def generate_demonstration_prompts_1round(user_ids, target_user, template_num, dataset_name, history):
    demo_templates = read_template('./templates/template_demonstration.txt')
    example_prompts = []
    prompt_str = "Here\'s some demonstration examples:\n"
    for user_id in user_ids:
        demo, target, _ = generate_zero_shot_prompt(str(user_id), demo_templates[template_num], dataset_name, history, 20)
        prompt_str += demo + "\n"
        prompt_str = prompt_str.replace('{target}', "[\'"+target+"\']")
    prompt_str += "Do you think the pattern of the demonstration examples are helpful for you to generate more accurate recommendation for {user}? If they are helpful, make some changes to improve your top-10 recommendation list to user {user}. If you don't think the example is helpful, then just keep the original answer. Still remember that the output format should be a python list of items' titles."
    # prompt_str += "Please try to make some changes to improve the recommendation to {user}. Still remember that the output format should be a python list of items' titles."
    prompt_str = prompt_str.replace('{user}', username[target_user])
        # example_prompts.append(prompt_str)
    return prompt_str

In [68]:
# task
# 0: 10 history & candidate random shuffled
# 1: 10 history
# 2: 10 history & history random shuffled & candidate random shuffled
# 3: 10 history user profile generation
# 4: 10 history in context & candidate random shuffled
# 5: 15 history & candidate random shuffled
# 6: 20 history & candidate random shuffled
# 7: 10 history & 5-shot examples & candidate random shuffled
def generate_prompt(sample_file, dataset_name, templates_file, history=10, task_list=[0,1,2,3], cand_len=20, base_file=None, users=None):
    with open(sample_file, 'r') as f:
        sample = eval(f.readline())
    samples = sorted(sample)
    # with open(f'results/{dataset}/sample_1k_all.txt', 'r') as f:
    #     content = f.readlines()
    # samples = [x[:-1] for x in content]
    samples = np.array(samples).astype(str)

    gen_base_input = []

    if base_file != None:
        f = open(base_file, 'r')
        content = f.read()
        f.close()
        gen_base_input = json.loads(content)
        if 0 in task_list:
            gen_input = []
            templates = read_template(templates_file[0])
            for i in range(len(samples)):
                template_num = random.randint(0, len(templates)-1)
                prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, cand_len, cand_list_id=eval(gen_base_input[i]['cand_list']))
                gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
            with open(f'results/{dataset_name}/test_history{history}_candidate{cand_len}_shuffle_sample.json', 'w') as file:
                json.dump(gen_input, file, indent=4)
    else:
        if 0 in task_list:
            templates = read_template(templates_file[0])
            for i in range(len(samples)):
                template_num = random.randint(0, len(templates)-1)
                prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, cand_len)
                gen_base_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
            with open(f'results/{dataset_name}/test_history{history}_candidate{cand_len}_shuffle_sample.json', 'w') as file:
                json.dump(gen_base_input, file, indent=4)
        else:
            raise ValueError
    if not gen_base_input:
        print('Baseline unexisted.')

    if 1 in task_list:
        gen_input = []
        templates = read_template(templates_file[2])
        for i in range(len(samples)):
            template_num = random.randint(0, len(templates)-1)
            prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, cand_len, cand_shuffle=False)
            gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
        with open(f'results/{dataset_name}/test_history{history}_candidate{cand_len}_sample.json', 'w') as file:
            json.dump(gen_input, file, indent=4)
    if 2 in task_list:
        gen_input = []
        templates = read_template(templates_file[2])
        for i in range(len(samples)):
            template_num = random.randint(0, len(templates)-1)
            prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, cand_len, his_shuffle=True, cand_list_id=eval(gen_base_input[i]['cand_list']))
            gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
        with open(f'results/{dataset_name}/test_history{history}_shuffle_candidate{cand_len}_shuffle_sample.json', 'w') as file:
            json.dump(gen_input, file, indent=4)
    if 3 in task_list:
        gen_input = []
        templates = read_template(templates_file[3])
        for i in range(len(samples)):
            template_num = random.randint(0, len(templates)-1)
            prompt, target = generate_user_profile_prompt(samples[i], templates[template_num], dataset_text, history)
            gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'template': template_num})
        with open(f'results/{dataset_name}/test_history{history}_generate_profile_sample_1k.json', 'w') as file:
            json.dump(gen_input, file, indent=4)
    if 4 in task_list:
        gen_input = []
        templates = read_template(templates_file[4])
        for i in range(len(samples)):
            template_num = random.randint(0, len(templates)-1)
            prompt, target, cand_list = generate_in_context_prompt(samples[i], templates[template_num], dataset_text, history, cand_len, cand_list_id=eval(gen_base_input[i]['cand_list']))
            gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
        with open(f'results/{dataset_name}/test_history{history}_candidate{cand_len}_in_context_sample.json', 'w') as file:
            json.dump(gen_input, file, indent=4)
    if 5 in task_list:
        gen_input = gen_base_input[:]
        templates = read_template(templates_file[8])
        for i in range(len(samples)):
            template_num = random.randint(0, len(templates)-1)
            prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, cand_len, cand_list_id=eval(gen_base_input[i]['cand_list']))
            if users == None:
                random_numbers = [random.randint(1, len(test_samples)) for _ in range(5)]
                while int(samples[i]) in random_numbers:
                    random_numbers = [random.randint(1, len(test_samples)) for _ in range(5)]
            else:
                random_numbers = [sample[x] for x in users[i]]
            few_shot_prompts = generate_demonstration_prompts_1round(random_numbers, samples[i], template_num, dataset_text)
            gen_input[i]['few_shot'] = [few_shot_prompts]
            # gen_input[i]['prompt'] = few_shot_prompts[0] + prompt
        with open(f'results/{dataset_name}/test_history{history}_candidate{cand_len}_few_shots_sample_{bool(users)}_1round_prompt.json', 'w') as file:
            json.dump(gen_input, file, indent=4)    

In [61]:
templates_file = ['../templates/template.txt', '../templates/template.txt', '../templates/template.txt', '../templates/template_user_profile_generate.txt', 
                  '../templates/template_in_context.txt', '../template_fewshot.txt']

In [78]:
generate_prompt(f'./results/{dataset}/sample.txt', dataset, templates_file, np.arange(7))

In [14]:
# retrieval model
import pandas as pd
# with open(f'./results/{dataset}/sample.txt', 'r') as f:
#     sample = eval(f.readline())
# samples = sorted(sample)
with open(f'results/{dataset}/sample_1k_all.txt', 'r') as f:
    content = f.readlines()
samples = [x[:-1] for x in content]
test_len = 1000
def read_sota_results(dataset, model, sample, history_max, shuffle):
    df = pd.read_csv(f'./dataset/{dataset}/rec-{model}-{history_max}-{shuffle}-all.csv', sep='\t')
    results_dict = df.set_index('user_id')['rec_items'].to_dict()
    # print(results_dict)
    id_results = list(map(lambda x: eval(results_dict[int(x)]), sample))
    return id_results
results_of_models = []
results_of_models.append(read_sota_results(dataset+'_100', 'SASRec', samples, 10, False))
results_of_models.append(read_sota_results(dataset+'_100', 'GRU4Rec', samples, 10, False))
results_of_models.append(read_sota_results(dataset+'_100', 'LightGCN', samples, 20, False))
results_of_models.append(read_sota_results(dataset+'_100', 'BPRMF', samples, 20, False))
def sample_mult_cands(model_results):
    cnt = 0
    ptr = 0
    pos = 0
    cands = []
    while cnt < 20:
        if model_results[ptr][pos] not in cands:
            cands.append(model_results[ptr][pos])
            cnt += 1
        if ptr % len(results_of_models) == len(results_of_models)-1:
            pos += 1
        ptr = (ptr+1)%len(results_of_models)
    return cands
cands_multi_model = [sample_mult_cands([x[i] for x in results_of_models]) for i in range(test_len)]
cands_multi_model = np.array(cands_multi_model).astype(str)

In [19]:
history = 10
samples = np.array(samples).astype(str)
gen_input = []
templates = read_template('./template.txt')
for i in range(len(samples)):
    template_num = random.randint(0, 4)
    prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, 20, cand_list_id=cands_multi_model[i])
    gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(list(cand_list)), 'template': template_num})
with open(f'results/{dataset}/test_history{history}_multi_candidate20_shuffle_sample.json', 'w') as file:
    json.dump(gen_input, file, indent=4)

In [7]:
def distillSummary(text):
    sentences = text.split('\n')
    overall = sentences[-1]
    return overall

In [32]:
# task:
# 0: history+profile
# 1: profile
def inject_profile_prompt(sample_file, model, templates_file, history, task_list=[0, 1], sample_cand_list=None):
    # with open(sample_file, 'r') as f:
    #     sample = eval(f.readline())
    # samples = sorted(sample)
    with open(f'results/{dataset}/sample_1k_all.txt', 'r') as f:
        content = f.readlines()
    samples = [x[:-1] for x in content]
    samples = np.array(samples).astype(str)
    with open(f'results/{dataset}/test_history{history}_generate_profile_sample_texts_{model}.json', 'r') as f:
        content = f.read()
        texts = json.loads(content)
    user2profile = [(x['user'], x['result']) for x in texts]
    user2profile = {k:v for (k, v) in user2profile}
    if 0 in task_list:
        gen_input = []
        templates = read_template(templates_file[0])
        for i in range(len(samples)):
            template_num = random.randint(0, 4)
            prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, 20, cand_list_id=sample_cand_list[i])
            prompt = prompt.replace('{user_profile}', "\'"+distillSummary(user2profile[int(samples[i])])+"\'")
            gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
        with open(f'results/{dataset}/test_history{history}_multi_candidate20_shuffle_profile_{model}_history_sample_1k.json', 'w') as file:
            json.dump(gen_input, file, indent=4)
    if 1 in task_list:
        gen_input = []
        templates = read_template(templates_file[1])
        for i in range(len(samples)):
            template_num = random.randint(0, 4)
            prompt, target, cand_list = generate_zero_shot_prompt(samples[i], templates[template_num], dataset_text, history, 20, cand_list_id=sample_cand_list[i])
            prompt = prompt.replace('{user_profile}', "\'"+distillSummary(user2profile[int(samples[i])])+"\'")
            gen_input.append({'prompt': prompt, 'target': target, 'user': samples[i], 'cand_list': str(cand_list), 'template': template_num})
        with open(f'results/{dataset}/test_history{history}_multi_candidate20_shuffle_profile_{model}_sample_1k.json', 'w') as file:
            json.dump(gen_input, file, indent=4)
    

In [ ]:
with open(f'./results/{dataset}/test_history{history}_candidate20_shuffle_sample.json', 'r') as f:
    content = f.read()
    prompts = json.loads(content)

In [18]:
profile_templates_file = ['./templates/template_profile_history.txt', './templates/template_user_profile.txt']

In [ ]:
model_name = 'gpt4'
inject_profile_prompt(f'./results/{dataset}/sample.txt', model_name, profile_templates_file, history, sample_cand_list=cands_multi_model)